In [1]:
import numpy as np
import pandas as pd
import os
import keras
from keras.layers import *
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras import regularizers
from sklearn.model_selection import train_test_split
import cv2
import matplotlib.pyplot as plt
import seaborn as sns
import glob

2022-12-24 00:58:50.875113: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-12-24 00:58:50.877584: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/usr/local/cuda/lib64
2022-12-24 00:58:50.877590: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
train_dir = 'data/train_small'
IMG_SIZE = 256,256

In [3]:
train_data = tf.keras.preprocessing.image_dataset_from_directory(   
    directory=train_dir,
    label_mode="categorical",
    color_mode="rgb",
    image_size=IMG_SIZE,
    seed=40,
    batch_size=256,
    validation_split=0.1,
    subset='training'
)

val_data = tf.keras.preprocessing.image_dataset_from_directory( 
    directory=train_dir,
    label_mode="categorical",
    color_mode="rgb",
    image_size=IMG_SIZE,
    seed=40,
    validation_split=0.1,
    batch_size=256,
    subset='validation',
    interpolation="bilinear"
)

Found 2698 files belonging to 27 classes.
Using 2429 files for training.
Found 2698 files belonging to 27 classes.
Using 269 files for validation.


2022-12-24 00:58:52.301964: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-24 00:58:52.302244: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/tienthinh/.local/lib/python3.8/site-packages/cv2/../../lib64::/usr/local/cuda/lib64
2022-12-24 00:58:52.302284: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/tienthinh/.local/lib/python3.8/site-packages/cv2/../../lib64::/usr/local/cuda/lib64
2022-12-24 00:58:52.302319: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load d

In [4]:
from tensorflow.keras.layers.experimental import preprocessing

data_augmentation = Sequential([
    keras.layers.GaussianNoise(stddev=0.5)
], name ="data_augmentation")

In [5]:
input_shape = (256, 256, 3)
# base_model = tf.keras.applications.ResNet50(include_top=False)
base_model = tf.keras.applications.EfficientNetB0(include_top=False)

base_model.trainable = False

# Tạo layer đầu vào
inputs = Input(shape=input_shape, name="input_layer")

x = data_augmentation(inputs)

# Cung cấp đầu vào cho base_model (sau khi tăng cường) và không huấn luyện nó 
x = base_model(x, training=False)

# Gộp các đặc trưng đầu ra của base model
x = GlobalAveragePooling2D(name="global_average_pooling_layer")(x)
# x = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001 )(x)

# x = Dense(256, kernel_regularizer = regularizers.l2(l = 0.016),activity_regularizer=regularizers.l1(0.006),
#                 bias_regularizer=regularizers.l1(0.006) ,activation='relu')(x)
x = Dense(256)(x)
x = Dropout(rate=.4)(x)  

# Đặt dense layer làm đầu ra
outputs = Dense(27, activation="softmax", name="output_layer")(x)

# Tạo mô hình với đầu vào và đầu ra
model = keras.Model(inputs, outputs)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 256, 256, 3)]     0         
                                                                 
 data_augmentation (Sequenti  (None, 256, 256, 3)      0         
 al)                                                             
                                                                 
 efficientnetb0 (Functional)  (None, None, None, 1280)  4049571  
                                                                 
 global_average_pooling_laye  (None, 1280)             0         
 r (GlobalAveragePooling2D)                                      
                                                                 
 dense (Dense)               (None, 256)               327936    
                                                                 
 dropout (Dropout)           (None, 256)               0     

In [6]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=.05),loss='categorical_crossentropy', metrics=['accuracy'])

In [10]:
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0.1, patience=30, verbose=1)
callbacks_list = [early_stopping]

In [11]:
model_hist = model.fit(train_data,validation_data = val_data, epochs = 50, callbacks=callbacks_list)

Epoch 1/50
10/10 [==============================] - 30s 3s/step - loss: 1.1935 - accuracy: 0.9559 - val_loss: 3.8892 - val_accuracy: 0.9219
Epoch 2/50
10/10 [==============================] - 30s 3s/step - loss: 2.2853 - accuracy: 0.9424 - val_loss: 5.4494 - val_accuracy: 0.8996
Epoch 3/50
10/10 [==============================] - 30s 3s/step - loss: 1.8935 - accuracy: 0.9448 - val_loss: 5.0518 - val_accuracy: 0.8810
Epoch 4/50
10/10 [==============================] - 30s 3s/step - loss: 2.0136 - accuracy: 0.9399 - val_loss: 8.4486 - val_accuracy: 0.8736
Epoch 5/50
10/10 [==============================] - 30s 3s/step - loss: 2.4749 - accuracy: 0.9411 - val_loss: 7.2257 - val_accuracy: 0.9331
Epoch 6/50
10/10 [==============================] - 30s 3s/step - loss: 1.9621 - accuracy: 0.9535 - val_loss: 5.4731 - val_accuracy: 0.9331
Epoch 7/50
10/10 [==============================] - 30s 3s/step - loss: 2.0153 - accuracy: 0.9588 - val_loss: 7.0578 - val_accuracy: 0.8959
Epoch 8/50
10/10 [==

In [13]:
model.save('model27.h5')